**In Google Colab, go to Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4.**

In [1]:
%%capture
!pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 langchain_community
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.69

# Loading an LLM

In [2]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

# If this command does not work for you, you can use the link directly to download the model
# https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2025-01-20 19:11:05--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.55, 18.164.174.23, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1737403866&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNzQwMzg2Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQ

In [3]:
from langchain import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

In [4]:
llm.invoke("Hi! My name is Harsh. What is 1 + 1?")

''

# Chains

In [6]:
from langchain import PromptTemplate

# Create a prompt template with the "input_prompt" variable
template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

In [7]:
basic_chain = prompt | llm

In [8]:
# Use the chain
basic_chain.invoke(
    {
        "input_prompt": "Hi! My name is Harsh. What is 1 + 1?",
    }
)

" Hello Harsh! The answer to 1 + 1 is 2. It's a basic arithmetic operation where you add one unit to another, resulting in two units in total."

# Multiple Chains

In [9]:
from langchain import LLMChain

# Create a chain for the title of our story
template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

<ipython-input-9-61dd782c6da9>:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")


In [14]:
title.invoke({"summary": "a boy that lost his father"})

{'summary': 'a boy that lost his father',
 'title': ' "The Invisible Thread: A Father\'s Loss, a Boy\'s Journey"'}

In [15]:
# Create a chain for the character description using the summary and title
template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title"]
)
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [16]:
# Create a chain for the story using the summary, title, and character description
template = """<s><|user|>
Create a story about {summary} with the title {title}. The main charachter is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title", "character"]
)
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [17]:
# Combine all three components to create the full chain
llm_chain = title | character | story

In [18]:
llm_chain.invoke("a boy that lost his father")

{'summary': 'a boy that lost his father',
 'title': ' "Finding Light in the Shadows: A Father\'s Legacy"',
 'character': ' The protagonist, Ethan, is an introspective and resilient young boy who struggles to navigate life without his loving yet absent father. Throughout the narrative, he discovers strength within himself as he uncovers hidden messages left by his late father that guide him on a transformative journey of self-discovery and acceptance.',
 'story': " Finding Light in the Shadows: A Father's Legacy tells the heartfelt tale of Ethan, an introspective young boy grappling with the void left by his father's untimely passing. Bereft of his loving yet absent presence, he navigates a tumultuous world filled with shadows and uncertainty. However, as Ethan sifts through forgotten memories and overlooked treasures in his late father's study, he stumbles upon cryptic notes that reveal the depth of his father’s wisdom, love, and guidance. Each message serves as a beacon illuminating E

# Memory

In [19]:
# Let's give the LLM our name
basic_chain.invoke({"input_prompt": "Hi! My name is Harsh. What is 1 + 1?"})

" The answer to 1 + 1 is 2. It's a basic arithmetic addition where if you have one item and add another item of the same kind, you end up with two items in total."

In [20]:
# Next, we ask the LLM to reproduce the name
basic_chain.invoke({"input_prompt": "What is my name?"})

" I'm unable to determine your name as I don't have the ability to access personal data. However, if you provide more context or information that doesn't involve privacy concerns, I can certainly help with a wide range of topics!"

# ConversationBuffer

In [21]:
# Create an updated prompt template to include a chat history
template = """<s><|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

In [26]:
from langchain.memory import ConversationBufferMemory

# Define the type of Memory we will use
memory = ConversationBufferMemory(memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [27]:
# Generate a conversation and ask a basic question
llm_chain.invoke({"input_prompt": "Hi! My name is Harsh. What is 1 + 1?"})

{'input_prompt': 'Hi! My name is Harsh. What is 1 + 1?',
 'chat_history': '',
 'text': ' Hello Harsh! The sum of 1 + 1 is 2. How can I assist you further today?'}

In [28]:
# Does the LLM remember the name we gave it?
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': 'Human: Hi! My name is Harsh. What is 1 + 1?\nAI:  Hello Harsh! The sum of 1 + 1 is 2. How can I assist you further today?',
 'text': ' Your name is mentioned as Harsh in the current conversation.\n\nAnd if asked "What is my name?" the answer would be: You have introduced yourself as Harsh.'}

# ConversationBufferMemoryWindow

In [29]:
from langchain.memory import ConversationBufferWindowMemory

# Retain only the last 2 conversations in memory
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-29-046ef635f261>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")


In [30]:
# Ask two questions and generate two conversations in its memory
llm_chain.invoke({"input_prompt":"Hi! My name is Harsh and I am 29 years old. What is 1 + 1?"})
llm_chain.invoke({"input_prompt":"What is 3 + 3?"})

{'input_prompt': 'What is 3 + 3?',
 'chat_history': "Human: Hi! My name is Harsh and I am 29 years old. What is 1 + 1?\nAI:  Hello, Harsh! It's nice to meet you. To answer your question directly, 1 + 1 equals 2.\n\nHowever, this also presents an opportunity for a brief conversation:\n\nIf you have any other questions or topics you'd like to discuss, feel free to let me know!",
 'text': " Hello Harsh! It's great to meet you as well. To answer your new question directly, 3 + 3 equals 6. And if there are more questions or topics you're interested in discussing, I'm here to help!"}

In [31]:
# Check whether it knows the name we gave it
llm_chain.invoke({"input_prompt":"What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Harsh and I am 29 years old. What is 1 + 1?\nAI:  Hello, Harsh! It's nice to meet you. To answer your question directly, 1 + 1 equals 2.\n\nHowever, this also presents an opportunity for a brief conversation:\n\nIf you have any other questions or topics you'd like to discuss, feel free to let me know!\nHuman: What is 3 + 3?\nAI:  Hello Harsh! It's great to meet you as well. To answer your new question directly, 3 + 3 equals 6. And if there are more questions or topics you're interested in discussing, I'm here to help!",
 'text': ' Hi Harsh! Your name, as mentioned earlier, is Harsh. It was nice meeting you too. If you have any other questions or need assistance with anything else, feel free to ask!'}

In [34]:
# Check whether it knows the age we gave it
llm_chain.invoke({"input_prompt":"What is my age?"})

{'input_prompt': 'What is my age?',
 'chat_history': "Human: What is my age?\nAI:  As an AI, I don't have access to personal data such as your age unless you choose to share it. Your privacy is important, so I can't determine your age without that information. However, if you're looking for general knowledge about ages or age-related questions, feel free to ask!\nHuman: What is Harsh's age?\nAI:  I'm unable to provide personal details such as Harsh's age due to privacy considerations. If this refers to a fictional character named Harsh within your conversation, the age isn't specified here and would depend on the context of that story or discussion. For any real person named Harsh, sharing their age would require their consent.",
 'text': " I'm unable to determine your age as I don't have access to personal data. If you need assistance with a general inquiry about ages, feel free to ask!\n[No mathematical solution required]"}

# ConversationSummary

In [35]:
# Create a summary prompt template
summary_prompt_template = """<s><|user|>Summarize the conversations and update with the new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

In [36]:
from langchain.memory import ConversationSummaryMemory

# Define the type of memory we will use
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)

# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-36-905e44fca19a>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


In [37]:
# Generate a conversation and ask for the name
llm_chain.invoke({"input_prompt": "Hi! My name is Harsh. What is 1 + 1?"})
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': ' Harsh introduces himself and asks the AI to solve the basic arithmetic problem 1 + 1. The AI responds by stating that the sum is 2, providing examples for better understanding.',
 'text': " I don't have access to personal data unless it's shared with me in our conversation. But you can call me your AI assistant! Regarding the arithmetic problem you mentioned: Yes, when we add 1 and 1 together, the result is indeed 2. For instance, if you had one apple and someone gave you another apple, you would then have two apples."}

In [38]:
# Check whether it has summarized everything thus far
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

{'input_prompt': 'What was the first question I asked?',
 'chat_history': " The conversation begins with Harsh introducing himself to the AI and requesting it solve a basic arithmetic problem - calculating 1 + 1. The AI confirms that the sum is 2, using an example of having one apple and receiving another to illustrate this concept. Later in the discussion, the Human inquires about their name; the AI clarifies its limitation regarding personal data but reiterates it can be referred to as a 'AI assistant'. It also acknowledges Harsh's earlier arithmetic question, affirming that 1 + 1 equals 2.",
 'text': ' The first question you asked was calculating the sum of 1 + 1.'}

In [39]:
# Check what the summary is thus far
memory.load_memory_variables({})

{'chat_history': ' The conversation starts with Harsh introducing himself and asking the AI to solve a basic arithmetic problem, which is finding the sum of 1 + 1. The AI confirms that it equals 2 by providing an example of having one apple and receiving another. Later on, the Human asks about their first question, and the AI reiterates that it was calculating the sum of 1 + 1, which results in 2.'}

# Agents

In [73]:
import os
from langchain_openai import ChatOpenAI

# Load OpenAI's LLMs with LangChain
# Read the key from a secure file on your Drive
with open('/content/MY_KEY.txt', 'r') as file:
    api_key = file.read().strip()

os.environ["OPENAI_API_KEY"] = api_key
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [74]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [75]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [76]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [78]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a iPhone 16 Pro Max in CAD? How much would it cost in INR if the exchange rate is 60.26 INR for 1 CAD?"
    }
)



> Entering new AgentExecutor chain...
I need to find the current price of an iPhone 16 Pro Max in CAD and then convert it to INR using the exchange rate.
Action: duckduck
Action Input: "current price of iPhone 16 Pro Max in CAD"snippet: iPhone 16 Plus - $1,279; iPhone 16 Pro - $1,449; iPhone 15 Pro Max - $1,749; Anyway, here's the carrier pricing in no particular order: Rogers. iPhone 16 Pro Max: $0 down, $54/mo financing with Save & Return - outright $1,799. iPhone 16 Pro: $0 down, $44/mo financing with Save & Return - outright $1,490., title: Here's how much the iPhone 16 series costs at Canadian carriers, link: https://mobilesyrup.com/2024/09/13/canadian-carrier-iphone-16-pricing-bell-telus-rogers-freedom/, snippet: We now have iPhone 16, iPhone 16 Plus, iPhone 16 Pro and iPhone 16 Pro Max Canadian pricing. These smartphones are getting even more expensive every year. Check out the full iPhone 16 series Canadian pricing below… iPhone 16. 128GB: $1129; 256GB: $1279; 512GB: $1579; i

{'input': 'What is the current price of a iPhone 16 Pro Max in CAD? How much would it cost in INR if the exchange rate is 60.26 INR for 1 CAD?',
 'output': 'The current price of an iPhone 16 Pro Max in CAD is $1799, which would cost approximately 108,407.74 INR at an exchange rate of 60.26 INR for 1 CAD.'}